In [4]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
import pickle


file_list = []
class_list = []

DATADIR = "training_set"

# All the categories you want your neural network to detect
CATEGORIES = ['bridge','childs','downwarddog','mountain','plank','seatedforwardbend','tree','trianglepose','warrior1','warrior2']

# The size of the images that your neural network will use
IMG_SIZE = 250

# Checking or all images in the data folder
for category in CATEGORIES :
	path = os.path.join(DATADIR, category)
	for img in os.listdir(path):
		img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)

training_data = []

def create_training_data():
	for category in CATEGORIES :
		path = os.path.join(DATADIR, category)
		class_num = CATEGORIES.index(category)
		for img in os.listdir(path):
			try :
				img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
				new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
				training_data.append([new_array, class_num])
			except Exception as e:
				pass

create_training_data()

random.shuffle(training_data)

X = [] #features
y = [] #labels

for features, label in training_data:
	X.append(features)
	y.append(label)
'''
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

X = X/255.0
y = np.array(y)
'''

'\nX = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)\n\nX = X/255.0\ny = np.array(y)\n'

In [0]:
from PIL import Image
import numpy as np 
from matplotlib import pyplot as plt 
import os
import cv2
import random
import pickle
from tqdm import tqdm
#pose = Image.open('training_set//bridge//File1.jpg').convert('LA')

categories = ['bridge','childs','downwarddog','mountain','plank','seatedforwardbend','tree','trianglepose','warrior1','warrior2']
datadir = 'training_set'
im_size = 250

training_data = []

for category in tqdm(categories):
    path = os.path.join(datadir,category)
    class_num = categories.index(category) 
    for im in os.listdir(path):
        try:
            im_arr = Image.open(os.path.join(path,im)).convert('L')
            new_im_arr = im_arr.resize((im_size,im_size),Image.ANTIALIAS)
            #print(type(new_im_arr))
            training_data.append([np.array(new_im_arr),class_num])
        except:
            pass
random.shuffle(training_data)

X = [] #features 
y = [] # labels 


for feat, label in training_data:
    X.append(feat)
    y.append(label)

print(len(X))
print(len(y))

X = np.array(X).reshape(-1, im_size, im_size, 1)
print(X[1].shape)

In [0]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt



model = Sequential()
# 3 convolutional layers
model.add(Conv2D(32, (3, 3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.30))

# 2 hidden layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.30))

model.add(Dense(128))
model.add(Activation("relu"))
# The output layer with 13 neurons, for 13 classes
model.add(Dense(10))
model.add(Activation("softmax"))

# Compiling the model using some basic parameters
model.compile(loss="sparse_categorical_crossentropy",
				optimizer="adam",
				metrics=["accuracy"])

# Training the model, with 40 iterations
# validation_split corresponds to the percentage of images used for the validation phase compared to all the images

history = model.fit(X, y, batch_size=32, epochs=40, validation_split=0.1)


# Saving the model
model_json = model.to_json()
with open("model.json", "w") as json_file :
	json_file.write(model_json)

model.save_weights("model.h5")
print("Saved model to disk")

model.save('CNN.model')
# Printing a graph showing the accuracy changes during the training phase


In [0]:
print(history.history.keys())

plt.plot(history.history['val_accuracy'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')


In [0]:
import cv2
import tensorflow as tf

CATEGORIES = ['bridge','childs','downwarddog','mountain','plank','seatedforwardbend','tree','trianglepose','warrior1','warrior2']

datadir = 'test_set'
catagory = 'bridge'
file1 = 'File10.jpg'

def prepare(file):
    IMG_SIZE = 250
    img_array = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    
#model = tf.keras.models.load_model("CNN.model")

image = prepare(os.path.join(catagory,file1))
prediction = model.predict([image])
prediction = list(prediction[0])
print(CATEGORIES[prediction.index(max(prediction))])